In [6]:
import numpy as np
import random
import datetime
import time
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import hdbscan
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
import pandas as pd
import seaborn as sns

import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

from ember_utils import *
from ember_model import *
from ember_pjr_utils import *


def min_count_cluster(clusterLabels, uniqueLabels):
    
    min_ = float('inf')
    for l in uniqueLabels:
        if l != -1:
            curr_l = np.where(clusterLabels == l)
            min_ = min(min_, len(curr_l))
    return min_


def get_cluster_samples(data, clusterLabels,\
                        uniqueLabels, eachClusterSamples):
    
    samples = []
    
    for l in uniqueLabels:
        if l != -1:
            curr_l = np.where(clusterLabels == l)
            curr_l_samples = list(data[curr_l])
            
            if len(curr_l_samples) <= eachClusterSamples:
                curr_l_samples = curr_l_samples
            else:
                curr_l_samples = random.sample(curr_l_samples, eachClusterSamples)
            #print(f'curr_l_samples {len(curr_l_samples)}')
            for s in curr_l_samples:
                samples.append(s)
                
    #print(f'len(samples) {len(samples)}')
        
    return samples
    
    
    

def get_HDBScanBased_samples_V1(family_name, family_data, num_samples_per_malware_family):
    data_X = np.array(family_data)
    
    if len(data_X) > 1:
        # fit the model
        clf_hdbscan = hdbscan.HDBSCAN()
        clf_fit_hdbscan = clf_hdbscan.fit(data_X)
        clf_hdbscan_labels = clf_fit_hdbscan.labels_
        unique_hdbscan_labels = np.unique(clf_hdbscan_labels)

        anomalous_idx = np.where(clf_hdbscan_labels == -1)
        
        anomalous_samples = list(data_X[anomalous_idx])
        
        if len(anomalous_samples) >= num_samples_per_malware_family:
            replay_samples = random.sample(anomalous_samples, num_samples_per_malware_family)
        
        else:
            if len(unique_hdbscan_labels) == 1:
                replay_samples = anomalous_samples
            else:
                remaining_samples_to_pick = num_samples_per_malware_family - len(anomalous_samples)
                #min_samples_cluster = min_count_cluster(clf_hdbscan_labels, unique_hdbscan_labels)
                #print(f'unique_hdbscan_labels {unique_hdbscan_labels}')
                samples_per_cluster_equal = int(np.ceil(remaining_samples_to_pick/(len(unique_hdbscan_labels) - 1)))
                #samples_per_cluster_equal = min(samples_per_cluster_equal, min_samples_cluster)

                cluster_samples = get_cluster_samples(data_X, clf_hdbscan_labels,\
                                                      unique_hdbscan_labels, samples_per_cluster_equal)
                if len(anomalous_samples) != 0:
                    #print(f'{np.array(anomalous_samples).shape} - {np.array(cluster_samples).shape}')
                    replay_samples = np.concatenate((anomalous_samples,\
                                                     cluster_samples))
                else:
                    replay_samples = cluster_samples
    else:
        replay_samples = data_X
    
    return np.array(replay_samples)

def get_HDBScanBased_similar_samples(ifbased_similar_samples):
    
    similar_malware_samples = []
    
    clf_hdbscan = hdbscan.HDBSCAN()
    clf_fit_hdbscan = clf_hdbscan.fit(ifbased_similar_samples)
    unique_hdbscan_labels = np.unique(clf_fit_hdbscan.labels_)

    if len(unique_hdbscan_labels) == 1:
        similar_malware_samples = ifbased_similar_samples
    else:
        exemplars = clf_fit_hdbscan.exemplars_

        #print(selected_family_samples)
        for cluster in exemplars:
            for sample in cluster:
                similar_malware_samples.append(sample)


    return similar_malware_samples


def get_HDBScanBased_samples_V2(family_name, family_data, 
                             min_cluster_size,
                             min_samples,
                             num_samples_per_malware_family):
    data_X = np.array(family_data)
    
    if len(data_X) > min_samples:  
        # fit the model
        clf_hdbscan = hdbscan.HDBSCAN(min_cluster_size, min_samples)
        clf_fit_hdbscan = clf_hdbscan.fit(data_X)
        clf_hdbscan_labels = clf_fit_hdbscan.labels_
        unique_hdbscan_labels = np.unique(clf_hdbscan_labels)
        
        if len(unique_hdbscan_labels) == 1:
            if len(data_X) > num_samples_per_malware_family:
                replay_samples = random.sample(list(data_X), num_samples_per_malware_family)
            else:
                replay_samples = data_X
        else:
            replay_samples = []
            exemplars = clf_fit_hdbscan.exemplars_
            for cluster in exemplars:
                for sample in cluster:
                    replay_samples.append(sample)
            
            if len(replay_samples) < num_samples_per_malware_family:
                rest_samples_number = int(num_samples_per_malware_family - len(replay_samples))
                
                outlier_idx = np.where(clf_hdbscan_labels == -1)
                outlier_samples = list(data_X[outlier_idx])
                
                #print(rest_samples_number, len(outlier_samples))
                if rest_samples_number < len(outlier_samples):
                    
                    rest_samples = random.sample(outlier_samples, rest_samples_number)
                    replay_samples += rest_samples
            
            #print(f'family {family_name} clusters {len(exemplars)} samples {len(replay_samples)}')
    else:
        replay_samples = data_X
    
    return np.array(replay_samples)


def get_HDBScanBased_samples(family_name, family_data, 
                             min_cluster_size,
                             min_samples,
                             num_samples_per_malware_family):
    data_X = np.array(family_data)
    
    if len(data_X) > num_samples_per_malware_family:  
        # fit the model
        clf_hdbscan = hdbscan.HDBSCAN(min_cluster_size, min_samples)
        clf_fit_hdbscan = clf_hdbscan.fit(data_X)
        clf_hdbscan_labels = clf_fit_hdbscan.labels_
        unique_hdbscan_labels = np.unique(clf_hdbscan_labels)
        
        if len(unique_hdbscan_labels) == 1:
            replay_samples = random.sample(list(data_X), num_samples_per_malware_family)
        else:
            split_num_sample = int(num_samples_per_malware_family/2)
            
            
            outliers_indx = np.where(clf_hdbscan_labels == -1)
            
            if len(outliers_indx[0]) < split_num_sample:
                outlier_samples = data_X[outliers_indx]
            else:
                outlier_samples = data_X[outliers_indx]
                outlier_samples = random.sample(list(outlier_samples), split_num_sample)
                
            clusters_samples_indx = np.where(clf_hdbscan_labels != -1)
            
            if len(clusters_samples_indx[0]) < split_num_sample:
                clusters_samples = data_X[clusters_samples_indx]
            else:
                clusters_samples = data_X[clusters_samples_indx]
                clusters_samples = random.sample(list(clusters_samples), split_num_sample)
            
            replay_samples = np.concatenate((outlier_samples,\
                                                     clusters_samples))
            
            
    else:
        replay_samples = data_X
    
    return np.array(replay_samples)


def get_sample_count(dataDict):
    mal_count_ = []
    good_count = 0
    for k, v in dataDict.items():
        if k != 'goodware':
            mal_count_ += [len(v)]
        else:
            good_count += len(v)
            
    mal_count = np.sum(np.array(mal_count_))
    print(f'good {good_count} mal {mal_count}')
    
    return mal_count

def get_replay_samples_HDBScanBased(global_family_dict,
                                    min_cluster_size,
                                    min_samples,
                                    num_samples_per_malware_family):
    tmp_family_dict = defaultdict(list)
    pre_malware_samples = []
    
    num_exemplars = 0
    cnt = 0
    for k, v in global_family_dict.items():
        if k != 'goodware':
            cnt += 1
            selected_family_samples = get_HDBScanBased_samples(k, v, 
                                                               min_cluster_size,
                                                               min_samples,
                                                               num_samples_per_malware_family)
            #print(f'family {k} samples {len(selected_family_samples)}')
            tmp_family_dict[k] = list(selected_family_samples)
            num_exemplars += len(selected_family_samples)
            #print(selected_family_samples)
            for sample in selected_family_samples:
                pre_malware_samples.append(sample)
    
    print(f'pre_malware_samples {len(pre_malware_samples)}')
    if len(global_family_dict['goodware']) < len(pre_malware_samples):
        pre_goodware_samples = random.sample(global_family_dict['goodware'], len(global_family_dict['goodware']))
    else:
        pre_goodware_samples = random.sample(global_family_dict['goodware'], len(pre_malware_samples))
    
    tmp_family_dict['goodware'] = list(pre_goodware_samples)
    print(f'pre_goodware_samples {len(pre_goodware_samples)}')
    
    samples_to_replay = np.concatenate((np.array(pre_goodware_samples), np.array(pre_malware_samples)))
    labels_to_replay = np.concatenate((np.zeros(len(pre_goodware_samples)), np.ones(len(pre_malware_samples))))
    
    validate_mal_count = get_sample_count(tmp_family_dict)
    
    print(validate_mal_count == len(pre_malware_samples))
    
    
    return samples_to_replay, labels_to_replay, tmp_family_dict, num_exemplars



def make_family_based_dict(X_train, Y_train, Y_train_family, task_month, global_family_dict):
    count = 0
    for x_ind, x_sample in enumerate(X_train):
        count += 1
        #print(x_ind, Y_train[x_ind])

        if Y_train[x_ind] == 0:
            global_family_dict["goodware"].append(x_sample)
        if Y_train[x_ind] == 1:
            if Y_train_family[x_ind] == '':
                global_family_dict["others_family"].append(x_sample)
            else:
                global_family_dict[Y_train_family[x_ind]].append(x_sample)

    print(f'Task {task_month} and #-of new samples stored {count}')
    
    return global_family_dict



all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

data_dir = '../../month_based_processing_with_family_labels/'

 

patience = 5
replay_type = 'hdbscan'



num_exps = 1 #args.num_exps
num_epoch = 500 #args.num_epoch
batch_size = 6000 #args.batch_size
num_samples_per_malware_family = 500


min_cluster_size = 10
min_samples = 5

exp_type = 'HDBScanBased'

exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]

exemplars_stat = {}



cnt =  1    
for exp in exp_seeds:
    #try:
        start_time = time.time()
        use_cuda = True
        print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
        use_cuda = use_cuda and torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        torch.manual_seed(exp)

        model = Ember_MLP_Net()

        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)

        if torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            model = nn.DataParallel(model)

        model = model.to(device)
        print(f'Model has {count_parameters(model)/1000000}m parameters')    
        criterion = nn.BCELoss()    



        stored_global_family_dict = defaultdict(list)

        standardization = StandardScaler()
        standard_scaler = None
        
        all_mal_training_samples = []
        for task_month in range(len(all_task_months)):

            print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Round {cnt} ...')
            task_start = time.time()

            #task_month = task_month
            current_task = all_task_months[task_month]
            task_months = all_task_months[:task_month+1]
            print(f'Current Task {current_task} w/ {num_samples_per_malware_family} samples to Replay per Malware family.')


            model_save_dir = '../HDBScanBased_pjr_saved_model_' +\
                        str(exp_type) + '/HDBScanBased_PJR_replay_' +\
                        str(num_samples_per_malware_family) + '/' + str(current_task) + '/'
            create_parent_folder(model_save_dir)

            opt_save_path = '../HDBScanBased_pjr_saved_optimizer_' +\
                        str(exp_type) + '/HDBScanBased_PJR_replay_' +\
                        str(num_samples_per_malware_family) + '/' + str(current_task) + '/'
            create_parent_folder(opt_save_path)


            results_save_dir = './HDBScanBased_saved_results_' +\
                        str(exp_type) + '/HDBScanBased_PJR_replay_' +\
                        str(num_samples_per_malware_family) + '/' 
            create_parent_folder(results_save_dir)


            X_train, Y_train, Y_train_family = get_family_labeled_month_data(data_dir, current_task)
            X_test, Y_test, Y_test_family = get_family_labeled_task_test_data(data_dir, task_months, mlp_net=True)
            
            
            print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
            standard_scaler = standardization.partial_fit(X_train)

            X_train = standard_scaler.transform(X_train)
            X_test = standard_scaler.transform(X_test)

            X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
            X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
            
            
            mal_samples = np.where(Y_train == 1)
            all_mal_training_samples.append(len(mal_samples[0]))

            # to debug
            #X_train, Y_train, Y_train_family = X_train[:5000], Y_train [:5000], Y_train_family[:5000]
            #X_test, Y_test, Y_test_family = X_test[:50], Y_test[:50], Y_test_family[:50]

            
            if current_task == all_task_months[0]:
                num_replay_samples = 0
                stored_global_family_dict = make_family_based_dict(X_train,\
                           Y_train, Y_train_family,\
                            current_task, stored_global_family_dict)    
                
                _, _, stored_global_family_dict, num_exemplar_samples =\
                    get_replay_samples_HDBScanBased(stored_global_family_dict,
                                                    min_cluster_size,
                                                    min_samples,
                                                    num_samples_per_malware_family)
                
                exemplars_stat[current_task] = stored_global_family_dict   
                mal_count_from_dict = get_sample_count(stored_global_family_dict)
                print(f'task {current_task} exemplars stored {num_exemplar_samples}')
                
            else:

                stored_global_family_dict = make_family_based_dict(X_train, Y_train, Y_train_family,\
                                                                       current_task, stored_global_family_dict)
                
                X_train, Y_train, stored_global_family_dict, num_exemplar_samples =\
                    get_replay_samples_HDBScanBased(stored_global_family_dict,
                                                    min_cluster_size,
                                                    min_samples,
                                                    num_samples_per_malware_family)
                print(f'task {current_task} exemplars used {num_exemplar_samples}')
                
                exemplars_stat[current_task] = num_exemplar_samples
        
            print(f'Current Training - X_train {X_train.shape} ')
            '''
            print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
            standard_scaler = standardization.partial_fit(X_train)

            X_train = standard_scaler.transform(X_train)
            X_test = standard_scaler.transform(X_test)

            X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
            X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
            '''

            print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Training ...')
            task_training_time, epoch_ran, training_loss, validation_loss  =\
                                    training_early_stopping(model, model_save_dir, opt_save_path,\
                                    X_train, Y_train, X_test, Y_test, patience,\
                                    batch_size, device, optimizer, num_epoch,\
                                     criterion, replay_type, current_task, exp, earlystopping=True)

        
            model = Ember_MLP_Net()
            model = model.to(device)
            #load the best model for this task
            best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
            print(f'loading best model {best_model_path}')
            model.load_state_dict(torch.load(best_model_path))

            optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
            best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
            print(f'loading best optimizer {best_optimizer}')
            optimizer.load_state_dict(torch.load(best_optimizer))


            acc, rocauc = testing_aucscore(model, X_test, Y_test, batch_size, device)

            task_end = time.time()

            print(f'Task Elapsed time {(task_end - task_start)/60} mins.')    

            
            results_f = open(os.path.join(results_save_dir + 'hdbscan_10_5_stdBefore_' + str(num_samples_per_malware_family) + '_results.txt'), 'a')
            result_string = '{}\t{}\t{}\t{}\t{}\t{}\t\n'.format(current_task,epoch_ran, task_training_time, acc, rocauc, num_replay_samples)
            results_f.write(result_string)
            results_f.flush()
            results_f.close()


        end_time = time.time()
        cnt += 1
        print(f'Elapsed time {(end_time - start_time)/60} mins.')
        
    
    #except:
    #    exp_seeds += [random.randint(99999, 999999)]
    #    pass

#exemplar_file = './hdbscanExemplars_minCluster20.npz'    
#np.savez(exemplar_file, exemplars_stat)

Torch 1.6.0 CUDA 10.2
Model has 3.132161m parameters

2022-12-16 02:55:22 Round 1 ...
Current Task 2018-01 w/ 500 samples to Replay per Malware family.
X_train (55722, 2381) Y_train (55722,) Y_tr_family (55722,)
X_test (6192, 2381) Y_test (6192,) Y_te_family (6192,)
2022-12-16 02:55:23 Standardizing ...
Task 2018-01 and #-of new samples stored 55722
pre_malware_samples 15471
pre_goodware_samples 15471


  0%|          | 0/9 [00:00<?, ?it/s]

good 15471 mal 15471
True
good 15471 mal 15471
task 2018-01 exemplars stored 15471
Current Training - X_train (55722, 2381) 
2022-12-16 02:58:39 Training ...
Epoch 1 of 500


  0%|          | 0/9 [00:00<?, ?it/s]

Train Loss: 0.7018, Train Acc: 0.5301
Val Loss: 0.6839, Val Acc: 0.6293
Validation loss decreased (inf --> 0.683873).  Saving model ...
../HDBScanBased_pjr_saved_model_HDBScanBased/HDBScanBased_PJR_replay_500/2018-01/best_model_epoch_1.pt
../HDBScanBased_pjr_saved_optimizer_HDBScanBased/HDBScanBased_PJR_replay_500/2018-01/best_optimizer_epoch_1.pt
Epoch 2 of 500


...

Train Loss: 0.0990, Train Acc: 0.9645
Val Loss: 0.1667, Val Acc: 0.9365
EarlyStopping counter: 4 out of 5
Epoch 12 of 500


  0%|          | 0/13 [00:00<?, ?it/s]

Train Loss: 0.0978, Train Acc: 0.9644
Val Loss: 0.1678, Val Acc: 0.9375
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 1.351 minutes
loading best model ../HDBScanBased_pjr_saved_model_HDBScanBased/HDBScanBased_PJR_replay_500/2018-12/best_model_epoch_7.pt
loading best optimizer ../HDBScanBased_pjr_saved_optimizer_HDBScanBased/HDBScanBased_PJR_replay_500/2018-12/best_optimizer_epoch_7.pt


100%|██████████| 13/13 [00:02<00:00,  5.32it/s]

test accuracy 0.9434962272644043 and ROC-AUC 0.9436449744371973
Task Elapsed time 14.552315306663512 mins.
Elapsed time 103.76502691109975 mins.


In [ ]:
exemplars_samples = [12324, 20463, 24626, 31995, 36498, 41805, 48096, 53178, 62630, 78204, 86381, 93384]
exemplars_samples = np.array(exemplars_samples)
all_mal_training_samples = [29309, 28078, 18139, 24166, 19977, 22628, 23957, 19623, 33367,
       50742, 44994, 44987]
all_mal_training_samples = np.array(all_mal_training_samples)

In [ ]:

all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']
#k = 11

exemplars_count = []

for k in range(12):
    curr_task_samples = exemplars_stat[all_task_months[k]]
    #print(len(curr_task_samples))

    num_exemplars = []
    for x, y in curr_task_samples.items():
        if x != 'goodware':
            num_exemplars += [len(y)]
        else:
            #pass
            print(f'good {len(y)}')
    #print(num_exemplars)

    print(sum(num_exemplars))
    cum_num_exemplars = np.cumsum(np.sort(num_exemplars))
    exemplars_count.append(cum_num_exemplars)
    
#plt.figure(figsize=(6, 4))
#plt.plot(cum_num_exemplars, 'blue', linewidth=2, marker= '*');

In [ ]:
save_file_name = 'default_HDBSCAN_exemplars.pdf'

colors = ['black', 'blue', 'red', 'orange', 'purple', 'green',\
         'cyan', 'pink', 'lime', 'gray', 'goldenrod', 'navy']
mons = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.figure(figsize=(10, 6))
for ind, m in enumerate(exemplars_count):
    plt.plot(m, c = colors[ind], linewidth=4, marker= '*', markersize=6, label=mons[ind]);

    
    
plt.legend(loc='best', fontsize=16)
plt.ylabel("Cumulative #of Samples", size=20)
plt.xlabel("Number of Families", size=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.title('Exemplars Selection using default HDBSCAN.exemplars_', fontsize=20)
plt.grid(True, linestyle='--', linewidth=1.)

#plt.savefig('./figures/isolation_forest/' + save_file_name, dpi=1000, bbox_inches='tight', transparent=True);